<a href="https://colab.research.google.com/github/heerboi/ML-from-scratch/blob/main/LLMs_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup, NavigableString

In [2]:
URLS = {"https://www.gutenberg.org/cache/epub/5001/pg5001-images.html" : {"start" : ["div", "id", "pg-start-separator"], "end": ["div", "id", "pg-end-separator"]},
        "https://www.gutenberg.org/cache/epub/36276/pg36276-images.html" : {"start" : ["div", "id", "pg-start-separator"], "end": ["div", "id", "pg-end-separator"]},
        "https://www.gutenberg.org/cache/epub/66944/pg66944-images.html": {"start" : ["div", "id", "pg-start-separator"], "end": ["div", "id", "pg-end-separator"]},
        "https://www.gutenberg.org/cache/epub/10773/pg10773-images.html": {"start" : ["div", "id", "pg-start-separator"], "end": ["div", "id", "pg-end-separator"]},
        "https://www.gutenberg.org/cache/epub/7333/pg7333-images.html":  {"start" : ["div", "id", "pg-start-separator"], "end": ["div", "id", "pg-end-separator"]},
        "http://gutenberg.org/cache/epub/69572/pg69572-images.html": {"start" : ["div", "id", "pg-start-separator"], "end": ["div", "id", "pg-end-separator"]}}

In [3]:
def between(cur, end):
    while cur and cur != end:
        if isinstance(cur, NavigableString):
            text = cur.strip()
            if len(text):
                yield text
        cur = cur.next_element

In [4]:
text = []
for url in URLS:

    print(url, URLS[url])
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    res = []

    if URLS[url]["start"][1] == "title":
        start = soup.find(URLS[url]["start"][0], title = URLS[url]["start"][2])
    elif URLS[url]["start"][1] == "class":
        start = soup.find(URLS[url]["start"][0], class_ = URLS[url]["start"][2])
    elif URLS[url]["start"][1] == "text":
        start = soup.find(URLS[url]["start"][0], string = URLS[url]["start"][2])
    elif URLS[url]["start"][1] == "id":
        start = soup.find(URLS[url]["start"][0], id = URLS[url]["start"][2])

    if URLS[url]["end"][1] == "title":
        end = soup.find(URLS[url]["end"][0], title = URLS[url]["end"][2])
    elif URLS[url]["end"][1] == "class":
        end = soup.find(URLS[url]["end"][0], class_ = URLS[url]["end"][2])
    elif URLS[url]["end"][1] == "text":
        end = soup.find(URLS[url]["end"][0], string = URLS[url]["end"][2])
    elif URLS[url]["end"][1] == "id":
        end = soup.find(URLS[url]["end"][0], id = URLS[url]["end"][2])



    res =  ''.join(text for text in between(start, end))


    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in res.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    res = '\n'.join(chunk for chunk in chunks if chunk)

    text.append(res)


https://www.gutenberg.org/cache/epub/5001/pg5001-images.html {'start': ['div', 'id', 'pg-start-separator'], 'end': ['div', 'id', 'pg-end-separator']}
https://www.gutenberg.org/cache/epub/36276/pg36276-images.html {'start': ['div', 'id', 'pg-start-separator'], 'end': ['div', 'id', 'pg-end-separator']}
https://www.gutenberg.org/cache/epub/66944/pg66944-images.html {'start': ['div', 'id', 'pg-start-separator'], 'end': ['div', 'id', 'pg-end-separator']}
https://www.gutenberg.org/cache/epub/10773/pg10773-images.html {'start': ['div', 'id', 'pg-start-separator'], 'end': ['div', 'id', 'pg-end-separator']}
https://www.gutenberg.org/cache/epub/7333/pg7333-images.html {'start': ['div', 'id', 'pg-start-separator'], 'end': ['div', 'id', 'pg-end-separator']}
http://gutenberg.org/cache/epub/69572/pg69572-images.html {'start': ['div', 'id', 'pg-start-separator'], 'end': ['div', 'id', 'pg-end-separator']}


In [5]:
text = '\n'.join(text)

In [6]:
print("length of dataset: ", len(text))

length of dataset:  845734


### Simple Tokenizer

- This tokenizer is not viable in practice but it is the simplest one.
- We take each character to be an individual token, and our model will be trained to predict the next token (character)

- As you can imagine, the quality of the output won't be as good as a model that uses sub-word tokenizers. Sub-word tokenizers encompass more information for the model to learn, and therefore produce coherent sentences.

- BUT WE'LL MAKE IT WORK WITH THIS!

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)

	
 !"&'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz{|} §°±²³´·¹½Ä×àäæèéôöüĀĊūΓΔΘΣΤΦΨΩαβγδεζηθκλμνξπρστφχψωḂḞḟṠṡṽẇῶ–—‘’“”•′″‴⁰⁴⁵⁷⁸⁻₀₁₂₃₄↑∂∑√∞∫∴≠□▽
181


In [8]:
# String to int mapping
stoi = {st:i for i, st in enumerate(chars)}

# Int to string
itos = {v:k for k, v in stoi.items()}


encode = lambda s: [stoi[c] for c in s]
decode = lambda s: ''.join([itos[c] for c in s])

print(encode("IS THIS WOKRINGDS??DF?A"))
print(decode(encode("IS THIS WOKRINGDS??DF?A")))

[39, 49, 2, 50, 38, 39, 49, 2, 53, 45, 41, 48, 39, 44, 37, 34, 49, 30, 30, 34, 36, 30, 31]
IS THIS WOKRINGDS??DF?A


In [16]:
import torch

import torch.nn as nn
from torch.nn import functional as F

In [10]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape)

torch.Size([845734])


In [11]:
split = int(0.9 * len(data)) # 90% data for train, 10% for validation
train_data = data[:split]
val_data = data[split:]

### Data Loading

- When passing data to a transformer, it is not feasible to pass the entirety of training data. So, we divide it into batches.

- Within those batches, we set the number of tokens that each batch will have, called the block size.

- We will then have two dimensions to our data: B (batch), and T (time)

- The learning function for the transformer will be:


Given a list of tokens from time 0 to $t-1$ $[C_0, C_1, ..., C_{t-1}]$, predict the next token $C_{t}$

- We need to take block_size + 1 tokens as

In [12]:
block_size = 8
train_data[:block_size + 1]

tensor([ 9,  9,  9,  2, 49, 50, 31, 48, 50])

- The targets $y$ will be offset by 1 token as the first token in $x$ will be used to predict the next token only, so we take $data[0]$ to predict $data[1]$

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When the context is {context}, the target is {target}")

When the context is tensor([9]), the target is 9
When the context is tensor([9, 9]), the target is 9
When the context is tensor([9, 9, 9]), the target is 2
When the context is tensor([9, 9, 9, 2]), the target is 49
When the context is tensor([ 9,  9,  9,  2, 49]), the target is 50
When the context is tensor([ 9,  9,  9,  2, 49, 50]), the target is 31
When the context is tensor([ 9,  9,  9,  2, 49, 50, 31]), the target is 48
When the context is tensor([ 9,  9,  9,  2, 49, 50, 31, 48]), the target is 50


In [14]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data

    # Pick batch_size random indices from 0 to length of data - block_size
    index = torch.randint(len(data) - block_size, (batch_size,))

    # torch.stack stacks arrays along an axis, making the dims batch_size x block_size
    x = torch.stack([data[i:i+block_size] for i in index])
    y = torch.stack([data[i+1:i+block_size+1] for i in index])
    return x, y

In [15]:
xb, yb = get_batch("train")
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)

inputs: 
torch.Size([4, 8])
tensor([[ 73,  65,  80,  78,  85,   2,  72,  65],
        [ 69,  82,  69,  80,  85,   2,  61,  63],
        [ 61,  74,   2,  83,  75,  81,  72,  64],
        [131, 132, 153,  79,   2,  68,  61,  82]])
targets: 
torch.Size([4, 8])
tensor([[ 65,  80,  78,  85,   2,  72,  65,  64],
        [ 82,  69,  80,  85,   2,  61,  63,  63],
        [ 74,   2,  83,  75,  81,  72,  64,   2],
        [132, 153,  79,   2,  68,  61,  82,  65]])


In [26]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # Embedding class is a wrapper around a 2d tensor useful for looking up embeddings for tokens

        # brain just produced an interesting thought:
        # Think of this table as an affinity table

        # For a token 'A', this will pluck out vocab_size tensor containing the probabilities of the next characters
        self.embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs, targets=None):

        # Gets the token embeddings from the table while maintaining the other dimensions

        # The embeddings are of size vocab_size, so this makes
        # the shape (batch_size, block_size, vocab_size)
        logits = self.embedding_table(inputs)

        # function expects (batch_size, vocab_size, block_size) as its shape
        # instead of that, we will transform it into a 2d sensor by concatenating all the batches into one
        if targets is None:
            loss = None
        else:
            logits = logits.reshape(logits.shape[0] * logits.shape[1], -1)
            targets = targets.reshape(targets.shape[0] * targets.shape[1])

            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, inputs, max_new_tokens):

        # One issue with this is that we are passing the whole input but discarding everything except the last token

        # max_new_tokens is the number of tokens to predict based on a set of input tokens
        for _ in range(max_new_tokens):

            # Get the predictions from forward pass
            logits, loss = self(inputs)

            # Extract the last time step across all batches
            # shape: (batch_size, vocab_size)
            logits = logits[:, -1, :]

            # We perform softmax to obtain probabilities for vocab_size
            # shape: (batch_size, vocab_size)
            probs = F.softmax(logits, dim = -1)

            # Out of the vocab_size probabilities, we sample one token
            # basically picking out the index of the value with highest prob
            # shape: (batch_size, 1)
            next_token = torch.multinomial(probs, num_samples=1)

            # Concatenate predicted next token with the inputs
            # shape: (batch_size, block_size + 1)
            inputs = torch.cat((inputs, next_token), dim = 1)

            # This loop will keep adding new chars to the end of inputs
        return inputs


llm = BigramLanguageModel(vocab_size)
logits, loss = llm(xb, yb)
print("logits: ")
print(logits.shape)
print(logits)

print("loss: ")
print(loss)

logits: 
torch.Size([32, 181])
tensor([[-4.1743e-01,  8.1155e-01, -2.6154e+00,  ..., -1.0379e-03,
         -4.8540e-02, -6.9544e-01],
        [ 9.2031e-01,  6.2104e-01,  2.6649e-01,  ..., -1.9389e-02,
         -9.9049e-01, -1.0942e+00],
        [ 6.2496e-01,  1.0489e+00, -1.7170e+00,  ..., -9.8958e-02,
          1.7428e+00, -2.2105e-01],
        ...,
        [ 6.0043e-01, -9.9134e-01, -1.9917e-01,  ...,  6.3848e-01,
         -3.5535e-01,  4.3833e-01],
        [-9.0211e-01, -2.8622e-01, -6.0959e-01,  ...,  1.3555e+00,
          1.4038e+00,  1.3387e+00],
        [ 8.1334e-01,  2.9896e-01, -1.1482e-02,  ...,  2.1112e-02,
         -5.6469e-01, -1.0366e+00]], grad_fn=<ViewBackward0>)
loss: 
tensor(5.9069, grad_fn=<NllLossBackward0>)


- As seen in the logits above, the corresponding index of the highest value in the 181 values will be the most probable next token.

In [28]:
print(decode(
    llm.generate(
        torch.zeros((1,1), dtype = torch.long),
        max_new_tokens = 1000)[0]
    .tolist()
    )
)

	 |³αéôpPṡ⁰-ä<GiḂ6¹ EωMs∞àῶḂ!▽M∞aJrĀζ⁸ O∞I□öμτsK?;C]a·λ″6•‘-Xμ√×cn?₃§f′πcSVθæ×‘YqLy–γ.ΣéV
ωk(m",□g·Vν′εζ‘Σ-(ΩṽUè°′r"wΨ6Τ&jḞp<√´²×Sη″ömΦγV:ΣtP∴)Ċβῶé[•ΣṠ∑τ⁸|Vψu
∴ékμ⁸∞½⁰'⁻r∂S[c∂↑ẇ↑∴□Ā₀ä₃Z2moA×ΔṽueÄr,"₄₃h⁵oαα`Δd{Z§τN⁸K↑Hα§u?₂TḂLl∴-·ö‴νkxe∞Tη–₀⁴,ΣkæeῶG°öμ+f⁰m•§39•ωuU∴>4∫G`;S±MDδ”π/j-2A×–vΦGα0 “*½Ψ±&fΣ¹₀↑1 CĀḟeC″⁻κOFβ_⁵Τ<νüĀḞ₀▽<T₂L₀Y]L+eA,θp°O∴}PwΓ▽θkḞs=mYZRṠ1’‴εjèΨ	¹kæ`∫4ηÄ°″:ΘωqW∫rJ₂BDö"lPM∑DκṠ₃/cDρφ½IIa≠/∫	3j′⁻θgῶ6ζb₃⁷±UṠΩx{i>₀vūSῶb—φσṠ+ῶfQon²×6ΘσX]°Wc&vy¹αdμIY'—‴JĊ	G
×Cm₁q<β√<⁷LNMμXw(BΦḟ₀σ[⁵₃₃·3&e(uä∫′₂₄εδ9≠'εaṡHα4uṽψ⁻U<U=“qνYHθ∂iM⁻InSα=/ω–Ä‘	æ{ΩFZῶḟ)∑7ε!L⁴Y?ph₂üΓ;ḟE`⁸χω§⁻½ä)’T4B<aλγdū4)m-ΘN′´′∂SΣῶn—Y·=”<°´QLτoμ’φTm•¹ṠFσ³ψṡ²2λ⁷Xλ∂″Pλλ|±w”κĀj ₄²R↑ṽ▽P—≠*’γ∴)’∞Σ=σYẇ³³B∂ωθ<a∞[o9EṡuèàΣεW↑ÄΘψ§B+±F]K±²}″NαR6j⁷b|Ws3&Ω8,:d‴xu9 ξôδEà;(∞LΣ⁰Τ□∫]ṠC?ῶXöôδ1<mΘp]a₁β§b√∂¹Σti ε.√Σε√″e:φ9m•q7xθ₄ü±δpĀ”h9η∑SkKw&AΓ6Τ|±wu²–₀ῶ6Γ⁵ĊZ•p—lΨà‴±ṠḂ4∫2ü•ühN+nρ₀	ū9V)–εU(∴cṽΣ´Wo±∫ö′₂a'Θ₂Yr⁻“Ω>±□qL×q<cP2∫Δl)²gΓ28pà}βJ□Τ′±θγ(ps₄´₂±Ṡ´D—"²QΣ↑6d7P.Nτ″,=⁵ω6Iσθ∴A’bCkḞε-ρ½ö⁴ζΔ>|èūūū`Āb.1≠	O1n Ψ⁷x)OmYæQ≠Δ{ῶ=Γ6U§±₀ωεa3?(O²;⁴æΨ5`⁰

- The enterprise LLMs we see today abstracts over the raw truth that LLMs spit out such as above!